In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
herb = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/7_herb_compound_combined_common_phenotypic_effects.tsv")
herb_asso = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/7_2_learning_herb_compound_disease_association.tsv")
disease = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/8_gold_disease_feature.tsv")
disgenet = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/curated_gene_disease_associations.tsv", skiprows = 21)

In [9]:
print len(herb)
print len(herb['compID'].drop_duplicates())
print len(herb['umls_id'].drop_duplicates())
herb.head(1)

20442
143
1879


,compID,umls_id
0,1020.0,C0002170


In [10]:
disease.head(1)

,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,DisGeNet_geneId,DIsGeNet_score,DIsGeNet_geneName,DIsGeNet_sourceId
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100188812,0.120543,RLS3,CTD_human


In [11]:
print len(disease)
print len(disease['UMLS_concept_ID'].drop_duplicates())
print len(disease['OMIM_ID'].drop_duplicates())
print len(disease['DisGeNet_geneId'].drop_duplicates())
disease.head(1)

10015
265
235
3521


,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,DisGeNet_geneId,DIsGeNet_score,DIsGeNet_geneName,DIsGeNet_sourceId
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100188812,0.120543,RLS3,CTD_human


In [12]:
print len(disgenet)
print len(disgenet['diseaseId'].drop_duplicates())
print len(disgenet['geneId'].drop_duplicates())
disgenet['diseaseId']  = disgenet['diseaseId'].str.slice(start = 5)
disgenet.head(1)

32834
7607
9362


,diseaseId,geneId,score,geneName,description,diseaseName,sourceId,NofPmids,NofSnps
0,C0000737,3440,0.12,IFNA2,"interferon, alpha 2",Abdominal Pain,CTD_human,1,0


In [13]:
print len(herb_asso)
print len(herb_asso['compID'].drop_duplicates()) # disease association이 존재하는 herbal compound(이 compound들은 drug feature처리가 되어있음.)
print len(herb_asso['umlsID'].drop_duplicates()) # disease의 수
herb_asso.head(1)

6037
23
1775


,compID,effectType,evidence,phenID,reference,umlsID
0,5525.0,Association,Adenocarcinoma,912.0,CTD[D019905],C0001418


In [14]:
# herb_asso의 disease 중, feature를 가지고 있는 애들
k = herb_asso[herb_asso['umlsID'].isin(disgenet['diseaseId'])]
print len(k)
print len(k['compID'].drop_duplicates())
print len(k['umlsID'].drop_duplicates())

5134
23
1468


In [15]:
# disgenet 중에, herb_asso의 disease를 보기
c = disgenet[disgenet['diseaseId'].isin(herb_asso['umlsID'])]
print len(c['diseaseId'].drop_duplicates())
print len(c['geneId'].drop_duplicates())

1468
6626


In [16]:
# 둘이 join 함
combined = pd.merge(left = k, right = c, left_on = 'umlsID', right_on='diseaseId')
print len(combined)
print len(combined.drop_duplicates())
print len(combined.drop_duplicates(subset = ['compID', 'umlsID']))
print len(combined['compID'].drop_duplicates())
print len(combined['umlsID'].drop_duplicates())
print len(combined['diseaseId'].drop_duplicates())
print len(combined['geneId'].drop_duplicates())
combined.head(1)

159296
159296
5134
23
1468
1468
6626


,compID,effectType,evidence,phenID,reference,umlsID,diseaseId,geneId,score,geneName,description,diseaseName,sourceId,NofPmids,NofSnps
0,5525.0,Association,Adenocarcinoma,912.0,CTD[D019905],C0001418,C0001418,10232,0.126263,MSLN,mesothelin,Adenocarcinoma,CTD_human,5,0


In [17]:
# learning에 쓰이는 gold standard disease feature로 filter해야함.
h = combined[combined['geneId'].isin(disease['DisGeNet_geneId'])]
print len(h)
print len(h.drop_duplicates())
print len(h.drop_duplicates(subset = ['compID', 'umlsID']))
print len(h['compID'].drop_duplicates())
print len(h['umlsID'].drop_duplicates())
print len(h['geneId'].drop_duplicates())
h.head(1)

114687
114687
4911
23
1289
3248


,compID,effectType,evidence,phenID,reference,umlsID,diseaseId,geneId,score,geneName,description,diseaseName,sourceId,NofPmids,NofSnps
3,5525.0,Association,Adenocarcinoma,912.0,CTD[D019905],C0001418,C0001418,1029,0.180217,CDKN2A,cyclin-dependent kinase inhibitor 2A,Adenocarcinoma,CTD_human,105,0


In [18]:
h.columns.values

array(['compID', 'effectType', 'evidence', 'phenID', 'reference', 'umlsID',
       'diseaseId', 'geneId', 'score', 'geneName', 'description',
       'diseaseName', 'sourceId', 'NofPmids', 'NofSnps'], dtype=object)

In [19]:
filtered = h[['compID', 'effectType', 'evidence', 'phenID', 'reference', 'umlsID','diseaseId', 'geneId', 'score', 'geneName', 'description','diseaseName', 'sourceId']]
results = filtered[~filtered['umlsID'].isin(disease['UMLS_concept_ID'])]
print len(results)
print len(results.drop_duplicates())
print len(results.drop_duplicates(subset = ['compID', 'umlsID']))
print len(results.drop_duplicates(subset = 'compID'))
print len(results.drop_duplicates(subset = 'umlsID'))
print len(results.drop_duplicates(subset = 'geneId'))

76265
76265
4257
22
1149
2046


In [133]:
results.to_csv("/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/6_Herb/7_3_learning_herb_compound_disease_association_final.tsv", index=False)
results.to_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/7_3_learning_herb_compound_disease_association_final.tsv", index=False)

In [3]:
# learning에 쓰일 수 있는 형태로 만들기
herb_feature = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/7_herb_compound_combined_common_phenotypic_effects.tsv")
common_feature = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/7_2_gold_drug_feature_id.tsv")

In [4]:
disease_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/8_gold_standard_disease_binary_matrix.tsv", index_col=0)
drug_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/8_gold_standard_drug_binary_matrix.tsv", index_col = 0)

In [5]:
disease_mat.head(1)

,102300,102400,102500,103780,104130,104300,106400,107650,109100,109200,...,607631,607681,607682,607850,608088,608105,608217,608320,608583,608622
100188812,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
drug_feature = drug_mat.index
disease_feature = disease_mat.index
print len(drug_feature), len(disease_feature)

2276 3521


In [21]:
herb_disease_asso = results[['compID', 'umlsID']].drop_duplicates()
herb_disease_asso.index = range(len(herb_disease_asso))

In [22]:
print len(herb_disease_asso)
print len(herb_disease_asso.drop_duplicates(subset = 'compID'))
print len(herb_disease_asso.drop_duplicates(subset = 'umlsID'))
herb_disease_asso.head(1)


4257
22
1149


,compID,umlsID
0,5525.0,C0001418


In [23]:
print len(herb_feature)
print len(herb_feature.drop_duplicates(subset = 'compID'))
print len(herb_feature.drop_duplicates(subset = 'umls_id'))
herb_feature.head(1)

20442
143
1879


,compID,umls_id
0,1020.0,C0002170


In [37]:
f = lambda x: 1 if x else 0

cnt = 0
#arr = np.empty((0,5797), dtype=np.int64)
arr = np.hstack(('herb_compound_id', 'disease_id', drug_mat.index, disease_mat.index))
for index, row in herb_disease_asso.iterrows():
    compID = row['compID']
    umlsID = row['umlsID']
    
    temp_herb_compound = herb_feature[herb_feature.compID == compID]['umls_id']
    hc_flags = drug_mat.index.isin(temp_herb_compound)
    herb_compound_flags = map(f, hc_flags)
    
    temp_herb_disease = results[results.umlsID == umlsID]['geneId'].drop_duplicates()
    hd_flags = disease_mat.index.isin(temp_herb_disease)
    herb_disease_flags = map(f, hd_flags)
    
    feature = np.hstack([compID, umlsID, np.array(herb_compound_flags), np.array(herb_disease_flags)])
    arr = np.vstack([arr, feature])
    cnt = cnt + 1
    if cnt % 1000 == 0 :
        print "%d : " % cnt + time.strftime('%a %H:%M:%S')
print "end : %d : " % cnt + time.strftime('%a %H:%M:%S')
print cnt
print arr.shape

1000 : Fri 17:24:42
2000 : Fri 17:25:45
3000 : Fri 17:27:24
4000 : Fri 17:29:41
end : 4257 : Fri 17:30:23
4257
(4258, 5799)


In [38]:
df_arr = pd.DataFrame(data = arr[1:], columns = arr[0])
print len(df_arr)
df_arr.head(1)

4257


,herb_compound_id,disease_id,C0000737,C0000786,C0001824,C0002170,C0002453,C0002622,C0002792,C0002871,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
0,5525.0,C0001418,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df_arr.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/3_herb_asso_descriptor.tsv", sep='\t', index=False)
df_arr.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/3_herb_asso_descriptor.tsv", sep='\t', index=False)

In [ ]:
#이전버전

In [173]:
herb_compound_descriptor = pd.concat([herb_disease_asso, df_arr], axis=1)

In [175]:
herb_compound_descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/3_herb_asso_descriptor.tsv", sep='\t', index=False)
herb_compound_descriptor.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/3_herb_asso_descriptor.tsv", sep='\t', index=False)

In [177]:
print len(herb_compound_descriptor)
herb_compound_descriptor.head(1)

4257


,compID,umlsID,0,1,2,3,4,5,6,7,...,5787,5788,5789,5790,5791,5792,5793,5794,5795,5796
0,5525.0,C0001418,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
